In [2]:
import SharedArray as sa
import numpy as np
import pandas as pd

In [3]:
qintra_spreadavgmidp11 = sa.attach("qintra_spreadavgmidp11").reshape(-1, 6000)
qtwap = sa.attach("qtwap").reshape(-1, 6000)
qhigh = sa.attach("qhigh").reshape(-1, 6000)
qs_val_mv = sa.attach("qs_val_mv").reshape(-1, 6000)
qret10 = sa.attach("qret10").reshape(-1, 6000)
qintra_midprice10_ret_stdev = sa.attach("qintra_midprice10_ret_stdev").reshape(-1, 6000)
qintra_price5avgmidp11 = sa.attach("qintra_price5avgmidp11").reshape(-1, 6000)
qopen_moneyflow_pct_value_l = sa.attach("qopen_moneyflow_pct_value_l").reshape(-1, 6000)

In [4]:
a = np.array([1, 2, 3, 4])
np.diff(a, prepend=[0])

array([1, 1, 1, 1])

In [5]:
np.roll(a, 1)

array([4, 1, 2, 3])

In [12]:
np.nanvar(qintra_midprice10_ret_stdev, axis=1).reshape(-1, 1).repeat(6000).reshape(
    -1, 6000
)

array([[3.9236556e-07, 3.9236556e-07, 3.9236556e-07, ..., 3.9236556e-07,
        3.9236556e-07, 3.9236556e-07],
       [2.1579437e-07, 2.1579437e-07, 2.1579437e-07, ..., 2.1579437e-07,
        2.1579437e-07, 2.1579437e-07],
       [3.2096884e-07, 3.2096884e-07, 3.2096884e-07, ..., 3.2096884e-07,
        3.2096884e-07, 3.2096884e-07],
       ...,
       [1.4150245e-07, 1.4150245e-07, 1.4150245e-07, ..., 1.4150245e-07,
        1.4150245e-07, 1.4150245e-07],
       [1.3409537e-07, 1.3409537e-07, 1.3409537e-07, ..., 1.3409537e-07,
        1.3409537e-07, 1.3409537e-07],
       [1.7654948e-07, 1.7654948e-07, 1.7654948e-07, ..., 1.7654948e-07,
        1.7654948e-07, 1.7654948e-07]], dtype=float32)

In [13]:
factor = [
    np.add(qintra_spreadavgmidp11, 50),
    qintra_spreadavgmidp11 - np.mean(qintra_spreadavgmidp11),
    np.divide(qtwap, qhigh),
    qintra_spreadavgmidp11 - qs_val_mv - np.roll(qs_val_mv, 1),
    qret10,
    np.nanvar(qintra_midprice10_ret_stdev, axis=1)
    .reshape(-1, 1)
    .repeat(6000)
    .reshape(-1, 6000),
    qintra_price5avgmidp11,
    np.zeros_like(qintra_price5avgmidp11),
    np.minimum(qintra_spreadavgmidp11, 10),
    np.add(qopen_moneyflow_pct_value_l, qintra_spreadavgmidp11),
]

In [14]:
weights = np.array(
    [
        0.01935759658112865,
        -0.013828770619796444,
        0.018830056369921407,
        -0.020424429368565564,
        -0.015565638852459636,
        -0.03826804504806913,
        -0.016740935787083692,
        0.06776920156106174,
        0.05998665475668967,
        0.016451108364566647,
    ]
)

In [16]:
np.dot(weights, factor)

ValueError: shapes (10,) and (10,11664,6000) not aligned: 10 (dim 0) != 11664 (dim 1)

In [20]:
res = np.zeros_like(factor[0])
for i in range(10):
    res += weights[i] + np.nan_to_num(factor[i], 0, 0, 0)

In [22]:
res.shape

(11664, 6000)

In [23]:
res

array([[ 6.4265060e+01, -8.3574190e+09, -1.2342139e+10, ...,
         7.7567190e-02,  7.7567190e-02,  7.7567190e-02],
       [ 6.2481525e+01, -8.3574190e+09, -1.2342139e+10, ...,
         7.7567019e-02,  7.7567019e-02,  7.7567019e-02],
       [ 6.2179817e+01, -8.3574190e+09, -1.2342139e+10, ...,
         7.7567123e-02,  7.7567123e-02,  7.7567123e-02],
       ...,
       [ 6.4138420e+01, -7.8454303e+09, -1.4270228e+10, ...,
         7.7566944e-02,  7.7566944e-02,  7.7566944e-02],
       [ 6.4108963e+01, -7.8454303e+09, -1.4270228e+10, ...,
         7.7566937e-02,  7.7566937e-02,  7.7566937e-02],
       [ 6.2991024e+01, -7.8454303e+09, -1.4270228e+10, ...,
         7.7566974e-02,  7.7566974e-02,  7.7566974e-02]], dtype=float32)

In [25]:
y = np.load("/home/public2/share_yw/data/basic_info/RET1D.npy")

In [30]:
from scipy.stats import pearsonr


def batch_pearsonr(x, y):
    results = []
    for row in range(len(x)):
        x_row = x[row]
        y_row = y[row]
        mask = ~np.isnan(y_row)
        x_row = x_row[mask]
        y_row = y_row[mask]
        if len(x_row) > 0:
            ic = pearsonr(x_row, y_row)[0]
            results.append(ic)
    return results

In [31]:
y.shape

(729, 16, 6000)

In [32]:
ics = batch_pearsonr(res, y.reshape(-1, 6000))

In [35]:
np.mean(ics)

0.0015701212946634558